<a href="https://colab.research.google.com/github/IgorNascimentoDEV/1-Projeto_PUC/blob/main/projeto_PUC.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

**Análise de Dados do DataSus**

Projeto de desenvolvimento de Aplicação para Tratamento de Dados - Turma 01 - 2023/2, vamos explorar e analisar os dados sobre casos de Dengue em 2022 fornecidos pelo Sistema de Informações Hospitalares do Sistema Único de Saúde (DataSus). Nosso objetivo é obter insights valiosos a partir desses dados e extrair informações úteis para futuras tomadas de decisão e análises.

**1. Configuração do Ambiente**

Antes de começarmos, vamos garantir que todas as bibliotecas necessárias estejam instaladas e configuradas.

In [4]:
#blibioteca responsavel por fazer a manipulação dos dados
import pandas as pd
pd.set_option('display.max_columns', None)

#biblioteca responsavel pela criação de graficos.
import matplotlib.pyplot as plt
plt.style.use('seaborn-talk')
import warnings
warnings.filterwarnings("ignore")

**2. Carregamento dos Dados**

Nessa etapa vamos utilizar o Pandas pra ler nossa base de dados que está localizada no Driver e armazenar todos os valores do csv na variavel dados.

In [2]:
dados = pd.read_csv('/content/drive/MyDrive/Projeto PUC/DENGBR22.csv')

**3. Exploração Inicial dos Dados**

Vamos começar explorando os dados para entender sua estrutura e conteúdo.

In [ ]:
#Primeira visualização dos dados
dados.head()

In [ ]:
#Visalizando colunas e tipos
colunas = dados.columns

# Obtém o nome de todas as colunas e seus tipos
colunas_e_tipos = [(coluna, tipo) for coluna, tipo in zip(dados.columns, dados.dtypes)]

# Exibe as informações das colunas e tipos
for coluna, tipo in colunas_e_tipos:
    print(f"Coluna: {coluna}, Tipo: {tipo}")

In [ ]:
#Conta a quantidade de valores nulos (NaN) em cada coluna
dados.isna().sum()

In [ ]:
#Mostra a quantidade total de valores nulos de toda a base de dados
dados.isna().sum().sum()


--------------------------------------------------teste-------------------------

In [8]:
for col in dados.columns:
    print(col)

TP_NOT
ID_AGRAVO
DT_NOTIFIC
SEM_NOT
NU_ANO
SG_UF_NOT
ID_MUNICIP
ID_REGIONA
ID_UNIDADE
DT_SIN_PRI
SEM_PRI
ANO_NASC
NU_IDADE_N
CS_SEXO
CS_GESTANT
CS_RACA
CS_ESCOL_N
SG_UF
ID_MN_RESI
ID_RG_RESI
ID_PAIS
DT_INVEST
ID_OCUPA_N
FEBRE
MIALGIA
CEFALEIA
EXANTEMA
VOMITO
NAUSEA
DOR_COSTAS
CONJUNTVIT
ARTRITE
ARTRALGIA
PETEQUIA_N
LEUCOPENIA
LACO
DOR_RETRO
DIABETES
HEMATOLOG
HEPATOPAT
RENAL
HIPERTENSA
ACIDO_PEPT
AUTO_IMUNE
DT_CHIK_S1
DT_CHIK_S2
DT_PRNT
RES_CHIKS1
RES_CHIKS2
RESUL_PRNT
DT_SORO
RESUL_SORO
DT_NS1
RESUL_NS1
DT_VIRAL
RESUL_VI_N
DT_PCR
RESUL_PCR_
SOROTIPO
HISTOPA_N
IMUNOH_N
HOSPITALIZ
DT_INTERNA
UF
MUNICIPIO
TPAUTOCTO
COUFINF
COPAISINF
COMUNINF
CLASSI_FIN
CRITERIO
DOENCA_TRA
CLINC_CHIK
EVOLUCAO
DT_OBITO
DT_ENCERRA
ALRM_HIPOT
ALRM_PLAQ
ALRM_VOM
ALRM_SANG
ALRM_HEMAT
ALRM_ABDOM
ALRM_LETAR
ALRM_HEPAT
ALRM_LIQ
DT_ALRM
GRAV_PULSO
GRAV_CONV
GRAV_ENCH
GRAV_INSUF
GRAV_TAQUI
GRAV_EXTRE
GRAV_HIPOT
GRAV_HEMAT
GRAV_MELEN
GRAV_METRO
GRAV_SANG
GRAV_AST
GRAV_MIOC
GRAV_CONSC
GRAV_ORGAO
DT_GRAV
MANI_HEMOR
EP